<a href="https://colab.research.google.com/github/kashindra-mahato/SOC-support-system/blob/kashindra/url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import pandas as pd
pd.options.display.max_colwidth = 100
pd.set_option('display.precision', 3)

In [123]:
path = '/content/drive/MyDrive/Colab Notebooks/DATASET/url.csv'
url = pd.read_csv(path, delimiter=',')

In [124]:
url.head()

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad


In [125]:
url.label.unique()

array(['bad', 'good'], dtype=object)

In [126]:
url.count()

url      420464
label    420464
dtype: int64

In [127]:
url.loc[url['label'] == 'bad'].count()

url      75643
label    75643
dtype: int64

In [128]:
url.loc[url['label'] == 'good'].count()

url      344821
label    344821
dtype: int64

In [129]:
replace_dict = {'good':0, 'bad':1}
url1 = url.replace({'label': replace_dict})

In [130]:
url1.head()

,url,label
0,diaryofagameaddict.com,1
1,espdesign.com.au,1
2,iamagameaddict.com,1
3,kalantzis.net,1
4,slightlyoffcenter.net,1


In [131]:
url1.count()

url      420464
label    420464
dtype: int64

In [132]:
url1.label.unique()

array([1, 0])

###customized token generator

In [133]:
def getTokens(input_data):
  # get tokens after splitting by slash
  tokenBySlash = str(input_data.encode('utf-8')).split('/')
  allTokens = []
  for i in tokenBySlash:
    # get tokens after splitting by dash
    tokens = str(i).split('-')
    tokensByDot = []
    for j in range(0, len(tokens)):
      # get tokens afters splitting by dot
      tokens1 = tokens[j].split('.')
      tokensByDot = tokensByDot + tokens1
    allTokens = allTokens + tokens + tokensByDot
  # remove reduntant tokens
  allTokens = list(set(allTokens))
  if 'com' in allTokens:
    # removing .com 
    allTokens.remove('com')
  return allTokens

###training

In [139]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import random

In [163]:
def Train(data, mod):
  # converting into dataframe
  urls = pd.DataFrame(data)
  # converting into numpy array
  urls = np.array(urls)
  # shuffling
  random.shuffle(urls)

  # labels and corpus
  y = [d[1] for d in urls]
  corpus = [d[0] for d in urls]
  # Vectorizer with custom tokenizer
  vectorizer = TfidfVectorizer(tokenizer=getTokens)
  x = vectorizer.fit_transform(corpus)

  # train and test split
  x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
  
  # models
  if mod == 'logic':
    model = LogisticRegression()
    model.fit(x_train, y_train)
    print(model.score(x_test, y_test))
  
  if mod == 'sgd':
    model = SGDClassifier(loss='perceptron')
    model.fit(x_train, y_train)
    print(model.score(x_test, y_test))
  
  return model,vectorizer



In [145]:
lgs, vectorizer1 = Train(url,'logic')

0.9835539224430095


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [146]:
X_predict = ['wikipedia.com','google.com/search=faizanahad','pakistanifacebookforever.com/getpassword.php/','www.radsport-voggel.de/wp-admin/includes/log.exe','ahrenhei.without-transfer.ru/nethost.exe','www.itidea.it/centroesteticosothys/img/_notes/gum.exe']
x_predict = vectorizer1.transform(X_predict)
y_predict = lgs.predict(x_predict)

In [147]:
y_predict

array(['bad', 'good', 'good', 'bad', 'bad', 'bad'], dtype='<U4')

In [164]:
sgd, vectorizer2 = Train(url,'sgd')

0.9964444127335212


In [169]:
X_predict = ['https://en.wikipedia.org/wiki/Main_Page','google.com/search=faizanahad','www.radsport-voggel.de/wp-admin/includes/log.exe','ahrenhei.without-transfer.ru/nethost.exe','www.itidea.it/centroesteticosothys/img/_notes/gum.exe']
x_predict = vectorizer2.transform(X_predict)
y_predict = sgd.predict(x_predict)

In [170]:
y_predict

array(['good', 'good', 'bad', 'bad', 'bad'], dtype='<U4')

In [167]:
X_predict = ['https://fmovies.to/movie/justice-league-throne-of-atlantis-pwlq/1-full','https://en.wikipedia.org/wiki/DC_Animated_Movie_Universe']
x_predict = vectorizer2.transform(X_predict)
y_predict = sgd.predict(x_predict)

In [168]:
y_predict

array(['good', 'good'], dtype='<U4')